In [1]:
#import csv
import json
import requests
import pandas as pd
from tqdm import tqdm  # progress_apply
import postgis as pg
from pymeos import *

# from tqdm.notebook import tqdm
import asyncio
import aiohttp
# -----------------------
# CONFIGURATION
# -----------------------
SERVER_URL = "http://localhost:8080"
COLLECTION_ID = "ships"           # Change to your collection name


LIMIT_ROWS = None
BATCH_SIZE = 50


# based on PYMEOS example : https://github.com/MobilityDB/PyMEOS-Examples/blob/main/PyMEOS_Examples/AIS.ipynb
# -----------------------
# HELPER FUNCTIONS
# -----------------------

# rework the data using pandas and pymeos to get temporal geometry (trip)

In [2]:
tqdm.pandas()

pymeos_initialize()
# %%time
# get data, cleanup remove duplicates, wrong lat,long ///



ais = pd.read_csv(
    "./data/aisdk-2024-03-01.zip",
    usecols=["# Timestamp", "MMSI", "Latitude", "Longitude", "SOG"],
    nrows=100
)
ais.columns = ["t", "mmsi", "lat", "lon", "sog"]
# data preparation:  remove rows with no timestamps
ais = ais[ais["t"] != 0]
ais["t"] = pd.to_datetime(ais["t"], format='%d/%m/%Y %H:%M:%S')
ais = ais[ais["mmsi"] != 0]
ais = ais.drop_duplicates(["t", "mmsi"])
ais = ais[(ais["lat"] >= 40.18) & (ais["lat"] <= 84.17)]
ais = ais[(ais["lon"] >= -16.1) & (ais["lon"] <= 32.88)]
ais = ais[(ais["sog"] >= 0) & (ais["sog"] <= 1022)]
ais.dropna()
ais.head()

,t,mmsi,lat,lon,sog
0,2024-03-01,219000873,56.990910,10.304543,0.0
2,2024-03-01,219016683,56.800165,9.024933,0.0
3,2024-03-01,219000615,56.967093,9.224287,2.3
5,2024-03-01,219017664,56.974950,8.922530,0.0
6,2024-03-01,219002686,56.795143,8.863960,0.0


# Now, we will create the PyMEOS object representing
# the position and the SOG.

In [3]:
ais["instant"] = ais.progress_apply(
    lambda row: TGeogPointInst(
        point=(row["lon"], row["lat"]), timestamp=row["t"]),
    axis=1,
)
ais["sog"] = ais.progress_apply(
    lambda row: TFloatInst(value=row["sog"], timestamp=row["t"]), axis=1
)
# dropt the standard type columns
ais.drop(["lat", "lon"], axis=1, inplace=True)
ais.head()

100%|██████████| 54/54 [00:00<00:00, 8727.70it/s]


,t,mmsi,sog,instant
0,2024-03-01,219000873,0@2024-03-01 00:00:00+01,POINT(10.304543 56.99091)@2024-03-01 00:00:00+01
2,2024-03-01,219016683,0@2024-03-01 00:00:00+01,POINT(9.024933 56.800165)@2024-03-01 00:00:00+01
3,2024-03-01,219000615,2.3@2024-03-01 00:00:00+01,POINT(9.224287 56.967093)@2024-03-01 00:00:00+01
5,2024-03-01,219017664,0@2024-03-01 00:00:00+01,POINT(8.92253 56.97495)@2024-03-01 00:00:00+01
6,2024-03-01,219002686,0@2024-03-01 00:00:00+01,POINT(8.86396 56.795143)@2024-03-01 00:00:00+01


# Assembling Trips (MEOS Example)
# Now, we will create the trajectory (TGeogPointSeq) and
# the SOG evolution (TFloatSeq) for every ship (identified by the mmsi)
# using the instants we have created.
# ::::::::::::::::::::::::::::stopped here , time limit

In [4]:

trajectories = (
    ais.groupby("mmsi")
    .aggregate(
        {
            "instant": lambda x: TGeogPointSeq.from_instants(x, upper_inc=True),
            "sog": lambda x: TFloatSeq.from_instants(x, upper_inc=True),
        }
    )
    .rename({"instant": "trajectory"}, axis=1)
)
trajectories["distance"] = trajectories["trajectory"].apply(
    lambda t: t.length())
trajectories.head()

,trajectory,sog,distance
mmsi,,,
211417590,[POINT(12.675237 54.524345)@2024-03-01 00:00:0...,[0.6@2024-03-01 00:00:00+01],0.0
211609560,[POINT(8.43944 55.01645)@2024-03-01 00:00:00+01],[0@2024-03-01 00:00:00+01],0.0
211648000,[POINT(7.201623 55.999482)@2024-03-01 00:00:00...,[7.3@2024-03-01 00:00:00+01],0.0
211718360,[POINT(15.656923 55.791803)@2024-03-01 00:00:0...,[8.1@2024-03-01 00:00:00+01],0.0
211796430,[POINT(9.036022 54.472213)@2024-03-01 00:00:00...,[0@2024-03-01 00:00:00+01],0.0


#### Create collection ships

In [ ]:
collection_metadata = {
    "title": "ships",
    "description": "Collection of ship trajectories",
    "updateFrequency": 1,
    "itemType": "movingfeature"  # OGC-compliant default
}

response = requests.post(
    "http://localhost:8080/collections",
    data=json.dumps(collection_metadata),
    headers={'Content-Type': 'application/json'}
)

if response.status_code == 201:
    print("Collection 'ships' created successfully!")
    print("Location:", response.headers.get('Location'))
else:
    print(f"Failed to create collection: {response.status_code} - {response.text}")


Collection 'ships' created successfully!
Location: /collections/ships


# ::::::::::::::::::::::::::::::stopped here , time limit
# Storing in MobilityDB: so they use PYMEOS but here use server.py- susceptible d'etre modifié!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# https://stackoverflow.com/questions/51699817/python-async-post-requests


In [8]:

def clean_up_resources():
 
    try:
        pymeos_finalize() 
        print("MEOS resources cleaned up successfully.")
    except Exception as e:
        print(f"Error during resource cleanup: {str(e)}")

server_url = 'http://localhost:8080/collections/{collectionId}/items'

collection_id = 'ships'


for index, row in trajectories.iterrows():
    try:
    
        post_data = {
            "type":"Feature",
            "id": row.name, 
            "temporalGeometry":json.loads(row["trajectory"].as_mfjson())  
        }

        
        json_data = json.dumps(post_data)
# post 
        response = requests.post(
            server_url.format(collectionId=collection_id), 
            data=json_data, 
            headers={'Content-Type': 'application/geo+json'}
        )

     
        if response.status_code == 200:
            print(f"Successfully posted feature {row.name}")
        else:
            print(f"Failed to post feature {row.name}: {response.status_code} - {response.text}")
    
    except requests.exceptions.RequestException as e:

        print(f"Request error for feature {row.name}: {str(e)}")
    
    except Exception as e:

        print(f"Unexpected error for feature {row.name}: {str(e)}")

clean_up_resources()


Failed to post feature 211417590: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 211609560: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 211648000: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 211718360: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 211796430: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 215760000: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 218801000: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 219000615: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 219000742: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 219000836: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 219000873: 201 - {"type": "FeatureCollection", "features": []}
Failed to post feature 219001431: 201 - {"type": "Feat

: 

### Retrieve all items

In [7]:

# Example bbox and datetime interval
params = {
    "x1": 139.757, 
    "y1": 35.627,
    "x2": 139.758,
    "y2": 35.628,
    "subTrajectory": "true",
    "dateTime": "2011-07-14T22:00:00Z,2011-07-15T02:00:00Z",
    "limit": "1"
}
response = requests.get(server_url, params=params)

print("Status code:", response.status_code)
try:
    data = response.json()
    print(json.dumps(data, indent=2))
except json.JSONDecodeError:
    print("Response was not valid JSON:")
    print(response.text)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

### Delete collection

In [ ]:
delete_response = requests.delete("http://localhost:8080/collections/ships")
print(delete_response.status_code, delete_response.text)